In [85]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
import pickle

In [361]:
def rmNegativeValRows(df):
       return df[(df >= 0).all(axis=1)]

def fitGaussian(data):
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    print("matrix and rank")
    print(cov.shape)
    print(np.linalg.matrix_rank(cov))
    pdfVals = multivariate_normal.pdf(data,mean=mean,cov=cov)
    print("PDFVals")
    print(pdfVals)
    likelihood = np.prod(pdfVals)
    
    ###
    logData = np.log(data)
    logMean = np.mean(logData, axis=0)
    logCov = np.cov(logData, rowvar=False)
    logPdfVals = multivariate_normal.pdf(logData,mean=logMean,cov=logCov)
    print(logPdfVals)
    logLikelihood = np.prod(logPdfVals)
    return(mean,cov,likelihood,
           logMean,logCov,logLikelihood)

In [332]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def scaleData(Xtrn,Xtst):
    scaler = StandardScaler()
    scaler.fit(Xtrn)
    XtrnScaled = scaler.transform(Xtrn)
    XtstScaled = scaler.transform(Xtst) 
    return(XtrnScaled,XtstScaled)

def minMaxScale(data):
    scaler = MinMaxScaler((0.1,1))
    scaler.fit(data)
    scaled = scaler.transform(data)
    return scaled

In [343]:
def transformData1(df):
    timeCols = [col for col in df.columns if col.endswith('ime')]
    for col in timeCols:
        df = df.ix[df[col] >= 0]
        df[col] = np.log(df[col] + 1e-20)
    return df
def transformData2(df):
    timeCols = [col for col in df.columns if col.endswith('ime')]
    for col in timeCols:
        df = df.ix[df[col] >= 0]
        df[col] = np.log(df[col] + 1e-20)
    (scaled,_) = scaleData(df,df)
    return scaled

def transformData3(df):
    timeCols = [col for col in df.columns if col.endswith('ime')]
    non_timeCols = [col for col in df.columns if not(col.endswith('ime'))]
    for col in timeCols:
        df = df.ix[df[col] >= 0]
        df[col] = np.log(df[col] + 1e-20)
    (scaled,_) = scaleData(df[non_timeCols],df[non_timeCols])
    return scaled

def rmNegativeTimes(df):
    timeCols = [col for col in df.columns if col.endswith('ime')]
    for col in timeCols:
        df = df.ix[df[col] > 0]
    return df

In [293]:
def gaussianMixtureGridSearch(params,data):
    """takes a dictionary containing a grid of parameters, fits to data and returns corresponding 
    GridSearchCV object"""
    clf = GaussianMixture()
    grid_search = GridSearchCV(clf, param_grid=params, cv = 3, 
                                       verbose=2, n_jobs=-1)
    grid_search.fit(data)
    return grid_search

In [360]:
#filepath to all data
filepath = "../data/SATALL12S_data/SATALL12Sft_and_slv_times_with_vals.csv"
df = pd.read_csv(filepath)
print(len(df))
df_clean = rmNegativeTimes(df)
print(len(df_clean))
clean_data = np.array(df_clean)[:,1:]

1614
703


/Users/luiscosta/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [359]:
#potential transformations
data_clean1 = transformData1(df) #log the runtimes, leave everything else as is
data_clean2   = transformData2(df) #log the runtimes, then standard scale everything.
data_clean3   = transformData3(df) #log the runtimes, then standard scale only non-runtime columns.

In [5]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV

In [326]:
params = {'n_components' : np.linspace(20,30,11).astype(int),
                'covariance_type' : ['full','tied','diag','spherical'],
                'reg_covar': np.logspace(-10, 1, 11)
                }
grid_search = gaussianMixtureGridSearch(params, data_clean3)

Fitting 3 folds for each of 484 candidates, totalling 1452 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1452 out of 1452 | elapsed:  4.6min finished
/Users/luiscosta/miniconda3/envs/myenv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [328]:
data_pos = np.array(rmNegativeValRows(df))
data_clean = data_pos[:,1:]

In [329]:
data_pos

array([], shape=(0, 157), dtype=float64)

In [327]:
grid_search.best_estimator_.lower_bound_

-107.17469003868563

In [285]:
grid_search.best_estimator_

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                means_init=None, n_components=29, n_init=1,
                precisions_init=None, random_state=None,
                reg_covar=0.005011872336272735, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [242]:
models = []

In [281]:
models.append(grid_search.best_estimator_)

In [282]:
models

[GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                 means_init=None, n_components=23, n_init=1,
                 precisions_init=None, random_state=None, reg_covar=0.001,
                 tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                 weights_init=None),
 GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                 means_init=None, n_components=27, n_init=1,
                 precisions_init=None, random_state=None,
                 reg_covar=0.005011872336272735, tol=0.001, verbose=0,
                 verbose_interval=10, warm_start=False, weights_init=None),
 GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                 means_init=None, n_components=25, n_init=1,
                 precisions_init=None, random_state=None,
                 reg_covar=0.005011872336272735, tol=0.001, verbose=0,
                 verbose_interval=10, warm_start=False, weight

In [351]:
results = all_dataGS.cv_results_
mean_scores = results['mean_test_score']

In [318]:
grid_searchMM = gaussianMixtureGridSearch(params,(np.log(minMaxScale(data_clean))))

ValueError: Found array with 0 sample(s) (shape=(0, 156)) while a minimum of 1 is required by MinMaxScaler.

In [127]:
grid_searchMM.best_estimator_

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                means_init=None, n_components=28, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-10,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [352]:
mean_scores= mean_scores.reshape(tuple(len(params[key]) for key in params))
diag_mean_scores = mean_scores[:,2,:]
X, Y, Z = np.zeros((diag_mean_scores.size,1)), np.zeros((diag_mean_scores.size,1)), np.zeros((diag_mean_scores.size,1))
counter = 0
for comp_idx in range(diag_mean_scores.shape[0]):
    for reg_idx in range(diag_mean_scores.shape[1]):
        X[counter],Y[counter] = params['n_components'][comp_idx], params['reg_covar'][reg_idx]
        Z[counter] = diag_mean_scores[comp_idx,reg_idx]

In [356]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

fig = plt.figure()
ax = fig.gca(projection='3d')

X, Y = np.meshgrid(params['n_components'],params['reg_covar'])
ax.plot_surface( X, Y, diag_mean_scores.T)
ax.set_xlabel("number of components")
ax.set_ylabel("Covariance Matrix regularization strength")
ax.set_zlabel("score")
ax.set_zlim(-100, 0)

(-100, 0)

In [288]:
pickle_out = open("GaussianMixtureList.pickle","wb")
pickle.dump(models, pickle_out)
pickle_out.close()

In [286]:
models

[GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                 means_init=None, n_components=23, n_init=1,
                 precisions_init=None, random_state=None, reg_covar=0.001,
                 tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                 weights_init=None),
 GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                 means_init=None, n_components=27, n_init=1,
                 precisions_init=None, random_state=None,
                 reg_covar=0.005011872336272735, tol=0.001, verbose=0,
                 verbose_interval=10, warm_start=False, weights_init=None),
 GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                 means_init=None, n_components=25, n_init=1,
                 precisions_init=None, random_state=None,
                 reg_covar=0.005011872336272735, tol=0.001, verbose=0,
                 verbose_interval=10, warm_start=False, weight

In [287]:
pickle_out = open("GaussianMixtureSATALL.pickle","wb")
pickle.dump(all_dataGS.best_estimator_, pickle_out)
pickle_out.close()

In [346]:
all_dataGS.best_estimator_

GaussianMixture(covariance_type='tied', init_params='kmeans', max_iter=100,
                means_init=None, n_components=27, n_init=1,
                precisions_init=None, random_state=None,
                reg_covar=0.06309573444801943, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [347]:
all_dataGS.best_estimator_.lower_bound_

1.1580546896575699